<a href="https://colab.research.google.com/github/choiking10/ML-tutorial/blob/main/2016_Gatys_NeuralStyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Style Transfer

Code to run Neural Style Transfer from paper Image Style Transfer Using Convolutional Neural Networks.

## Reference Code

Official code : https://github.com/leongatys/PytorchNeuralStyleTransfer




In [6]:
import torch
from torch import nn
from collections import OrderedDict
from PIL import Image
from torch.autograd import Variable

import torchvision
from torchvision import transforms
from torch import optim

import torch
from torch import nn
from collections import OrderedDict
from PIL import Image
from torch.autograd import Variable

import torchvision
from torchvision import transforms
from torch import optim


class VGG(nn.Module):
    def __init__(self, pool='max'):
        super(VGG, self).__init__()
        self.ativ = nn.ReLU
        if pool == 'max':
            self.pooling = nn.MaxPool2d
        elif pool == "avg":
            self.pooling = nn.AvgPool2d
        else:
            assert True, "pooling layer must be selected in ['max', 'avg']"

        self.conv0 = self.create_layer([3, 64, 64])
        self.conv1 = self.create_layer([64, 128, 128])
        self.conv2 = self.create_layer([128, 256, 256, 256, 256])
        self.conv3 = self.create_layer([256, 512, 512, 512, 512])
        self.conv4 = self.create_layer([512, 512, 512, 512, 512])
        self.feature_extractor = {}
        self.feature = []

    def forward(self, x, layers: list):
        #extractor = FeatureExtractor(self, layers)
        #
        # x = self.conv0(x)
        # x = self.conv1(x)
        # x = self.conv2(x)
        # x = self.conv3(x)
        # x = self.conv4(x)

        # extractor.clear_hook()
        ret = [None for i in range(len(layers))]

        for name, module in filter(lambda n:  "." in n[0], self.named_modules()):
            x = module(x)
            if name in layers:
                ret[layers.index(name)] = x
        return list(ret)

    def create_layer(self, param):
        layer = []
        start = param[0]
        iter = enumerate(param)
        next(iter)
        for i, f in iter:
            layer.append((f"c{i-1}", nn.Conv2d(start, f, kernel_size=3, padding=1)))
            layer.append((f"r{i-1}", nn.ReLU(inplace=True)))
            start = f
        layer.append(("p", self.pooling(kernel_size=2, stride=2)))
        return nn.Sequential(OrderedDict(layer))

    def load_vgg_weight(self):
        import torchvision.models as models
        vgg = models.vgg19(pretrained=True)
        state_dict = []
        for (p_name, p_param), (v_name, v_param) in zip(self.named_parameters(), vgg.named_parameters()):
            state_dict.append((p_name, v_param))
        self.load_state_dict(OrderedDict(state_dict))

def preprocessing():
  transforms.Compose([transforms.Scale(img_size),
                      transforms.ToTensor(),
                      transforms.Lambda(lambda x: x[torch.LongTensor([2, 1, 0])]),
                      transforms.Normalize(mean=[0.40760392, 0.45795686, 0.48501961], #subtract imagenet mean
                                          std=[1, 1, 1]),
                      transforms.Lambda(lambda x: x.mul(255))
                      ])


def main():
    vgg = VGG("avg")
    vgg.load_vgg_weight()
    print(vgg)

